In [1]:
# import dependencies
import nltk
import json
import io
import gzip
import torch
import string
import random
#import jsonlines
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm_notebook
from functools import partial
#import mwparserfromhell

import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer

import guidedlda

In [2]:
pd.__version__

'0.25.3'

In [3]:
PATH_TO_DATA = "/scratch/nh1724/"

In [4]:
ls $PATH_TO_DATA

base_model_eta.pkl              results_lda_top.pth
base_model_guidedlda_all.pkl    results.pth
base_model_guidedlda.pkl        run-jupyter.sbatch
base_model.pkl                  slurm-6130271.out
en_outlinks_tokens_df.pkl       squad/
en_tokens_lem_stem_df.pkl       squad-QA-char/
features.pkl                    test/
graph_df.pkl                    text_embed_en.pkl
inference_project_teresa.ipynb  wikitext_tokenized_text_sections_outlinks_en.p*
Inference-topic-model/


In [5]:
# import importlib.util

# def module_from_file(module_name, file_path):
#     spec = importlib.util.spec_from_file_location(module_name, file_path)
#     module = importlib.util.module_from_spec(spec)
#     spec.loader.exec_module(module)
#     return module

# my_lib = module_from_file("my_lib", PATH_TO_FOLDER + "my_lib.py")


## Create dataframe

In [6]:
# wiki_df_links = pkl.load(open(PATH_TO_DATA + "en_links_df.pkl", "rb"))
# wiki_df_tokens = pkl.load(open(PATH_TO_DATA + "wikitext_en_tokenized.p", "rb"))

# assert (wiki_df_tokens.QID != wiki_df_links.QID).sum() == 0, "Ids do not match!"

# wiki_df_links.columns

# wiki_df_links["raw_tokens"] = wiki_df_tokens["raw_tokens"]
# wiki_df_links["tokens"] = wiki_df_tokens["tokens"]
# wiki_df_links.rename(columns={"links": "raw_outlinks", "links_cleaned": "outlinks"}, inplace=True)
# wiki_df = wiki_df_links[["QID", "title", "raw_outlinks", "outlinks", "raw_tokens", "tokens", "mid_level_categories"]]

# pkl.dump(wiki_df, open(PATH_TO_DATA + "en_outlinks_tokens_df.pkl", "wb"))

In [7]:
wiki_df = pkl.load(open(PATH_TO_DATA + "wikitext_tokenized_text_sections_outlinks_en.p", "rb"))

## LDA

In [8]:
import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(57)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/nh1724/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
def lemmatize_stemming(list_of_tokens):
    output = []
    for token in list_of_tokens:
        output.append(stemmer.stem(lemmatizer.lemmatize(token, pos='v')))
    return output

In [10]:
# test
lemmatize_stemming("I did doing procrastination wolves cats".split())

['i', 'do', 'do', 'procrastin', 'wolv', 'cat']

In [11]:
# del wiki_df["raw_tokens"]
wiki_df.tokens = wiki_df.tokens.apply(lemmatize_stemming)

In [12]:
wiki_df["tokens_lem_stem"] = wiki_df["tokens"]
wiki_df.head()

,QID,mid_level_categories,tokens,sections_tokens,raw_outlinks,outlinks,tokens_lem_stem
0,Q6199,"[History_And_Society.History and society, Hist...","[anarch, anti, authoritarian, anti, authoritar...","[etymology, terminology, definition, history, ...","[[[Anti-authoritarianism|anti-authoritarian]],...","[Anti-authoritarianism, political philosophy, ...","[anarch, anti, authoritarian, anti, authoritar..."
1,Q38404,"[STEM.Medicine, STEM.Biology, History_And_Soci...","[autism, development, disord, character, diffi...","[characteristics, social, development, communi...","[[[Psychiatry]], [[Interpersonal relationship|...","[Psychiatry, Interpersonal relationship, commu...","[autism, development, disord, character, diffi..."
2,Q101038,"[STEM.Physics, STEM.Space, History_And_Society...","[sunlight, relat, various, surfac, condit, alb...","[terrestrial, albedo, white, sky, black, sky, ...",[[[File:Albedo-e hg.svg|thumb|upright=1.3|The ...,"[File:Albedo-e hg.svg, diffuse reflection, sun...","[sunlight, relat, various, surfac, condit, alb..."
3,Q173,[Geography.Americas],"[alabama, alabama, nicknam, northern, flicker,...","[etymology, history, pre, european, settlement...","[[[Coat of arms of Alabama|Coat of arms]], [[N...","[Coat of arms of Alabama, Northern flicker, Di...","[alabama, alabama, nicknam, northern, flicker,..."
4,Q41746,"[Culture.People, Geography.Europe, History_And...","[date, three, zero, zero, bc, achill, kill, et...","[etymology, birth, early, years, names, hidden...",[[[File:Achilles fighting against Memnon Leide...,[File:Achilles fighting against Memnon Leiden ...,"[date, three, zero, zero, bc, achill, kill, et..."


In [13]:
#pkl.dump(wiki_df[["QID", "tokens_lem_stem"]], open(PATH_TO_DATA + "en_tokens_lem_stem_df.pkl", "wb"))


In [14]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(wiki_df.mid_level_categories)
print(f"Number of categories: {len(mlb.classes_)}, \n\n{mlb.classes_}")

Number of categories: 45, 

['Culture.Architecture' 'Culture.Arts' 'Culture.Broadcasting'
 'Culture.Crafts and hobbies' 'Culture.Entertainment'
 'Culture.Food and drink' 'Culture.Games and toys'
 'Culture.Internet culture' 'Culture.Language and literature'
 'Culture.Media' 'Culture.Music' 'Culture.People'
 'Culture.Performing arts' 'Culture.Philosophy and religion'
 'Culture.Sports' 'Culture.Visual arts' 'Geography.Africa'
 'Geography.Americas' 'Geography.Antarctica' 'Geography.Asia'
 'Geography.Bodies of water' 'Geography.Europe' 'Geography.Landforms'
 'Geography.Maps' 'Geography.Oceania' 'Geography.Parks'
 'History_And_Society.Business and economics'
 'History_And_Society.Education' 'History_And_Society.History and society'
 'History_And_Society.Military and warfare'
 'History_And_Society.Politics and government'
 'History_And_Society.Transportation' 'STEM.Biology' 'STEM.Chemistry'
 'STEM.Engineering' 'STEM.Geosciences' 'STEM.Information science'
 'STEM.Mathematics' 'STEM.Medicine' '

In [44]:
mlb.classes_[26:32]

array(['History_And_Society.Business and economics',
       'History_And_Society.Education',
       'History_And_Society.History and society',
       'History_And_Society.Military and warfare',
       'History_And_Society.Politics and government',
       'History_And_Society.Transportation'], dtype=object)

In [15]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(wiki_df, y, test_size=0.1, random_state=42) #all train, output features
#X_train.shape, X_test.shape

In [16]:
#to output features, use full set for X, y
X_train = wiki_df
y_train = y

In [17]:
#compute prior eta based on a subset of whole corpus
def init_eta(y, bow_corpus, dictionary, num_examples=1000):
    '''
    y: MultiLabelBinarizer array, (num_doc, num_topic)
    bow_corpus: bag of word corpus
    dictionary: word vocab
    num_examples: restrict on a smaller subset as initialization
    '''
    eta = np.zeros(y.T.shape)
    y = y.astype(int) #ensure integer type
    for i in range(num_examples):
        key_idx = np.array(list(dict(bow_corpus[i]).keys()))
        counts = np.array(list(dict(bow_corpus[i]).values()))
        key_idx = key_idx.astype(int) #ensure integer type
        eta[np.ix_(y[i]==1,key_idx)] += counts
    return eta/np.sum(eta) #normalization

In [18]:
dictionary = gensim.corpora.Dictionary(X_train.tokens)
dictionary.filter_extremes(no_below=15, no_above=0.5,keep_n=y_train.shape[0]) #keep same shape as num of examples to initialize the prior (can improve)

In [19]:
bow_corpus = [dictionary.doc2bow(doc) for doc in X_train.tokens]

In [20]:
len(bow_corpus), len(dictionary)

(33823, 33823)

In [21]:
y_train.shape

(33823, 45)

In [22]:
eta0 = init_eta(y_train, bow_corpus,dictionary, num_examples=5000) #use small subset to guess seeded words

In [23]:
eta0.shape

(45, 33823)

## guidedLDA

In [24]:
#create seeded topic for GuidedLDA (list of list)
topic_words = []
n_top_words=10
for i in range(45): #45 topics
    topic_words_i = [dictionary[idx] for idx in np.array(dictionary)[np.argsort(eta0[i])][:-(n_top_words+1):-1]]
    topic_words.append(topic_words_i)

In [25]:
#show topic and top topic_words
df_topic = pd.DataFrame(topic_words)
df_topic = df_topic.T
df_topic.columns = mlb.classes_
df_topic.head(10)

,Culture.Architecture,Culture.Arts,Culture.Broadcasting,Culture.Crafts and hobbies,Culture.Entertainment,Culture.Food and drink,Culture.Games and toys,Culture.Internet culture,Culture.Language and literature,Culture.Media,...,STEM.Geosciences,STEM.Information science,STEM.Mathematics,STEM.Medicine,STEM.Meteorology,STEM.Physics,STEM.Science,STEM.Space,STEM.Technology,STEM.Time
0,build,art,televis,card,film,food,game,game,languag,music,...,island,librari,distribut,cell,ice,energi,system,star,system,ndash
1,hous,jpg,seri,bank,seri,water,player,internet,write,media,...,earth,univers,mean,human,temperatur,physic,theori,orbit,comput,american
2,design,file,show,debit,charact,product,chess,user,word,televis,...,nbsp,leibniz,statist,effect,earth,theori,nbsp,earth,engin,day
3,centuri,museum,film,stamp,anim,produc,play,blog,book,film,...,water,copyright,algorithm,diseas,snow,nbsp,scienc,space,oper,english
4,art,centuri,season,system,comic,plant,card,web,english,news,...,rock,carnegi,data,studi,satellit,electron,centuri,system,program,player
5,jpg,sculptur,broadcast,currenc,show,process,franklin,search,publish,game,...,miner,book,frac,system,atmospher,particl,color,nbsp,design,politician
6,file,bc,tv,knot,award,milk,film,network,film,public,...,geolog,columbia,sampl,caus,orbit,light,comput,observ,languag,actor
7,palac,collect,radio,reserv,releas,acid,video,video,centuri,blue,...,carbon,public,probabl,medic,nbsp,system,univers,sun,data,footbal
8,citi,british,big,feder,televis,drink,hand,googl,univers,broadcast,...,larg,dictionari,popul,organ,weather,forc,human,planet,power,singer
9,architectur,de,program,nickel,play,common,piec,irc,letter,advertis,...,metal,school,valu,psycholog,water,relat,natur,solar,nbsp,calendar


In [26]:
seed_topics = {}
for t_id, st in enumerate(topic_words):
    for word in st:
        seed_topics[dictionary.token2id[word]] = t_id

In [27]:
#GuidedLDA with seed words init
#model = guidedlda.GuidedLDA(n_topics=45, n_iter=100, random_state=7, refresh=20)

In [45]:
topic_map = {idx:0 for idx in range(16)}
topic_map.update({idx:1 for idx in range(16,26,1)})
topic_map.update({idx:2 for idx in range(26,32,1)})
topic_map.update({idx:3 for idx in range(32,45,1)})

In [47]:
len(topic_map)

45

In [50]:
#GuidedLDA with only 4 broad topics
seed_topics_top = {}
for idx, st in enumerate(topic_words):
    for word in st:
        top_id = topic_map[idx]
        seed_topics_top[dictionary.token2id[word]] = top_id

In [51]:
seed_topics_top

{1438: 1,
 2210: 0,
 915: 3,
 108: 3,
 1819: 0,
 3252: 0,
 2120: 0,
 3377: 0,
 120: 2,
 1815: 0,
 3328: 0,
 3507: 0,
 72: 1,
 128: 0,
 93: 0,
 180: 1,
 1327: 0,
 2591: 0,
 1281: 0,
 2121: 1,
 1678: 1,
 1889: 0,
 2744: 0,
 6147: 0,
 1429: 0,
 1220: 3,
 2935: 0,
 1843: 0,
 19433: 0,
 4142: 0,
 690: 3,
 173: 0,
 7184: 0,
 5371: 0,
 286: 0,
 9801: 0,
 2944: 0,
 1804: 3,
 6313: 0,
 4673: 0,
 1243: 0,
 536: 0,
 989: 0,
 1751: 3,
 1641: 2,
 2488: 0,
 1632: 3,
 1217: 0,
 2343: 0,
 8522: 3,
 5575: 0,
 134: 0,
 999: 1,
 7620: 3,
 5522: 0,
 3867: 0,
 4207: 0,
 1017: 0,
 3406: 0,
 1068: 0,
 6643: 0,
 19859: 0,
 4642: 0,
 4573: 0,
 2379: 0,
 8132: 0,
 14480: 0,
 1083: 3,
 772: 2,
 768: 0,
 89: 3,
 240: 3,
 3453: 0,
 733: 3,
 3944: 0,
 1138: 0,
 2325: 0,
 2381: 0,
 579: 3,
 1432: 0,
 1778: 0,
 2831: 0,
 2624: 0,
 2520: 0,
 1423: 0,
 1188: 0,
 164: 2,
 19651: 0,
 771: 2,
 756: 2,
 415: 0,
 540: 2,
 904: 0,
 3362: 0,
 4321: 0,
 4181: 0,
 12067: 0,
 706: 3,
 1580: 2,
 2166: 0,
 340: 3,
 2688: 1,
 2282:

In [52]:
model = guidedlda.GuidedLDA(n_topics=4, n_iter=100, random_state=7, refresh=20)

In [53]:
#create sparse input matrix from corpus
X = gensim.matutils.corpus2csc(bow_corpus,dtype=int).T

In [54]:
X.shape

(33823, 33823)

In [55]:
model.fit(X, seed_topics=seed_topics_top, seed_confidence=0.15)

INFO:guidedlda:n_documents: 33823
INFO:guidedlda:vocab_size: 33823
INFO:guidedlda:n_words: 43830236
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 100
/home/nh1724/.conda/envs/lda/lib/python3.6/site-packages/guidedlda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:guidedlda:<0> log likelihood: -443823432
INFO:guidedlda:<20> log likelihood: -385607858
INFO:guidedlda:<40> log likelihood: -384500082
INFO:guidedlda:<60> log likelihood: -384190903
INFO:guidedlda:<80> log likelihood: -384020308
INFO:guidedlda:<99> log likelihood: -383894968


In [56]:
#save model
pkl.dump(model, open(PATH_TO_DATA + "base_model_guidedlda_top.pkl", "wb"))

In [27]:
#load model
model = pkl.load(open(PATH_TO_DATA + "base_model_guidedlda_top.pkl", "rb"))

In [57]:
model.topic_word_.shape

(4, 33823)

In [58]:
#word topic assignment
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = [dictionary[t] for t in np.argsort(topic_dist)[:-(n_top_words+1):-1]]
    print(str(i), topic_words)

0 ['ndash', 'film', 'american', 'game', 'team', 'play', 'award', 'season', 'player', 'open']
1 ['citi', 'languag', 'india', 'area', 'district', 'region', 'popul', 'river', 'centuri', 'south']
2 ['war', 'govern', 'polit', 'parti', 'univers', 'countri', 'law', 'elect', 'forc', 'would']
3 ['system', 'nbsp', 'differ', 'product', 'water', 'process', 'type', 'exampl', 'common', 'effect']


In [30]:
X_train.shape

(33823, 7)

In [31]:
X_train.head()

,QID,mid_level_categories,tokens,sections_tokens,raw_outlinks,outlinks,tokens_lem_stem
0,Q6199,"[History_And_Society.History and society, Hist...","[anarch, anti, authoritarian, anti, authoritar...","[etymology, terminology, definition, history, ...","[[[Anti-authoritarianism|anti-authoritarian]],...","[Anti-authoritarianism, political philosophy, ...","[anarch, anti, authoritarian, anti, authoritar..."
1,Q38404,"[STEM.Medicine, STEM.Biology, History_And_Soci...","[autism, development, disord, character, diffi...","[characteristics, social, development, communi...","[[[Psychiatry]], [[Interpersonal relationship|...","[Psychiatry, Interpersonal relationship, commu...","[autism, development, disord, character, diffi..."
2,Q101038,"[STEM.Physics, STEM.Space, History_And_Society...","[sunlight, relat, various, surfac, condit, alb...","[terrestrial, albedo, white, sky, black, sky, ...",[[[File:Albedo-e hg.svg|thumb|upright=1.3|The ...,"[File:Albedo-e hg.svg, diffuse reflection, sun...","[sunlight, relat, various, surfac, condit, alb..."
3,Q173,[Geography.Americas],"[alabama, alabama, nicknam, northern, flicker,...","[etymology, history, pre, european, settlement...","[[[Coat of arms of Alabama|Coat of arms]], [[N...","[Coat of arms of Alabama, Northern flicker, Di...","[alabama, alabama, nicknam, northern, flicker,..."
4,Q41746,"[Culture.People, Geography.Europe, History_And...","[date, three, zero, zero, bc, achill, kill, et...","[etymology, birth, early, years, names, hidden...",[[[File:Achilles fighting against Memnon Leide...,[File:Achilles fighting against Memnon Leiden ...,"[date, three, zero, zero, bc, achill, kill, et..."


In [59]:
#document topics assignment
doc_topic = model.transform(X)

/home/nh1724/.conda/envs/lda/lib/python3.6/site-packages/guidedlda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):


In [60]:
pkl.dump(doc_topic, open(PATH_TO_DATA + "features_top.pkl", "wb"))

In [34]:
from sklearn.metrics import precision_recall_fscore_support

def get_metrics_dict(y_true, y_pred):
    """
    Help function that tests the model's performance on a dataset.
    """
    # macro precision, recall, f-score
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true,
        y_pred,
        average="macro"
    )
    # micro precision, recall, f-score
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true,
        y_pred,
        average="micro"
    )
    # combine all metrics in a dict
    dict_metrics = {
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_micro": precision_micro, 
        "recall_micro": recall_micro, 
        "f1_micro": f1_micro,
    }
    # round
    n_digits = 3
    dict_metrics = {
        metric_name: round(value, n_digits) 
        for metric_name, value in dict_metrics.items()
    }
    return dict_metrics

In [35]:
#threshold: each doc topic sum to 1, threshold topic w/ prob > 1/45
from copy import deepcopy
y_pred = deepcopy(doc_topic)
y_pred = np.where(y_pred>1/45, 1, 0)

In [ ]:
get_metrics_dict(y_train, y_pred)

In [59]:
#get X_test sparse input
bow_corpus_test = [dictionary.doc2bow(doc) for doc in X_test.tokens]

In [48]:
doc_topic_test = model.transform(X_test) #model.transform(X_train)

ValueError: Shape of passed values is (2, 23549), indices imply (3383, 7)

In [ ]:
#sparse input of X_test


In [ ]:
#save test features first
pkl.dump(doc_topic_test, open(PATH_TO_DATA + "features_test.pkl", "wb"))

In [ ]:
vocab = list(dictionary.values()) #list of terms in the dictionary
vocab_tf = [dict(i) for i in bow_corpus]
vocab_tf = list(pd.DataFrame(vocab_tf).sum(axis=0)) #list of term frequencies

In [ ]:
#save results for visz:https://github.com/vi3k6i5/GuidedLDA/issues/23
visz = {'topic_term_dists':topic_word,
        'doc_topic_dists':doc_topic,
        'doc_lengths': doc_lengths,
        'vocab':vocab, 
        'term_frequency':vocab_tf}

In [ ]:
pkl.dump(visz,open(PATH_TO_DATA + "visz.pkl", "wb"))

In [ ]:
#import visz data
data = pkl.load(open(PATH_TO_DATA + "visz.pkl", "rb"))
import pyLDAvis
# prepare the data
tef_vis_data = pyLDAvis.prepare(**data)

# this bit needs to be run after running the earlier code for reasons
pyLDAvis.display(tef_vis_data)

# save to HTML
pyLDAvis.save_html(tef_vis_data, "LDAvis.html")

In [ ]:
#quick check of results
for i in range(5): 
    print("Top Topic: {}; True Topic {}; Document: {}".format(doc_topic[i].argsort()[-5:][::-1],
                                                              np.where(y_train[i]==1)[0].tolist(),
                                                              ', '.join([dictionary.id2token[t] for t in np.argsort(X_train[i].toarray().flatten())[:-6:-1]])))

In [ ]:
#predictions
doc_topic_test = model.transform(X_test)

In [ ]:
#output feature vectors of doc-topic distribution for graphical NN usage
pkl.dump(np.concatenate((doc_topic,doc_topic_test),axis=0), open(PATH_TO_DATA + "features.pkl", "wb"))

In [59]:
# %install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
# %load_ext autotime

### LDA with eta0

In [60]:
from gensim import corpora, models
lda = models.LdaMulticore(bow_corpus, num_topics=45, id2word=dictionary, passes=2, workers=4, eta=eta0)

/home/nh1724/.conda/envs/lda/lib/python3.6/site-packages/gensim/models/ldamodel.py:1108: RuntimeWarning: invalid value encountered in multiply
  score += np.sum((self.eta - _lambda) * Elogbeta)
/home/nh1724/.conda/envs/lda/lib/python3.6/site-packages/gensim/models/ldamodel.py:1109: RuntimeWarning: invalid value encountered in subtract
  score += np.sum(gammaln(_lambda) - gammaln(self.eta))


In [61]:
import os
os.getcwd()

'/scratch/nh1724'

In [62]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
#lda.save(temp_file)
pkl.dump(temp_file, open(PATH_TO_DATA + "base_model_eta.pkl", "wb"))

In [63]:
for line,bag in zip(wiki_df.tokens[:5],bow_corpus[:5]):
    doc_topics = ['({}, {:.1%})'.format(topic, prob) for topic,prob in lda.get_document_topics(bag)]
    print('{} {}'.format(line[0:10], doc_topics))

['anarch', 'anti', 'authoritarian', 'anti', 'authoritarian', 'polit', 'philosophi', 'anarch', 'usual', 'place'] ['(4, 84.0%)', '(17, 6.7%)', '(20, 2.7%)', '(33, 6.2%)']
['autism', 'development', 'disord', 'character', 'difficulti', 'social', 'interact', 'communic', 'restrict', 'repetit'] ['(28, 58.1%)', '(33, 41.8%)']
['sunlight', 'relat', 'various', 'surfac', 'condit', 'albedo', 'mean', 'whiten', 'measur', 'diffus'] ['(0, 61.6%)', '(2, 31.8%)', '(7, 5.4%)', '(34, 1.1%)']
['alabama', 'alabama', 'nicknam', 'northern', 'flicker', 'yellowhamm', 'state', 'list', 'state', 'bird'] ['(4, 1.4%)', '(9, 15.6%)', '(12, 11.4%)', '(20, 12.6%)', '(23, 19.8%)', '(40, 26.4%)', '(41, 12.8%)']
['date', 'three', 'zero', 'zero', 'bc', 'achill', 'kill', 'ethiopian', 'king', 'memnon'] ['(3, 54.7%)', '(22, 8.2%)', '(42, 36.7%)']


In [66]:
y[5]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

## Starting exploration

In [ ]:
titles_set = set(wiki_df.title)
title_lower_set = set(wiki_df.title.apply(lambda x: x.lower()))
len(titles_set), len(title_lower_set)

In [ ]:
def overlap_with_lowercase_titles(links_list):
    count = 0
    for link in links_list:
        if link.lower() in title_lower_set:
            count += 1
    return count

def overlap_with_titles(links_list):
    count = 0
    for link in links_list:
        if link in titles_set:
            count += 1
    return count

wiki_df["links_cleaned"] = wiki_df.links.apply(clean_links)
wiki_df["count_overlap"] = wiki_df.links_cleaned.apply(overlap_with_titles)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.median(wiki_df.count_overlap), np.mean(wiki_df.count_overlap)

In [ ]:
(wiki_df.count_overlap == 0).sum() / len(wiki_df.count_overlap)

In [ ]:
median, mean = np.median(wiki_df.count_overlap), np.mean(wiki_df.count_overlap)

plt.figure(figsize=(10,6))
plt.hist(wiki_df.count_overlap, bins=150)
y_lim = 6e3
plt.vlines(median, 0, y_lim, colors="orange", linestyles='--', label=f"Median {median}")
plt.vlines(mean, 0, y_lim, colors="orange", linestyles='solid', label=f"Mean {mean:.3}")
plt.xlim(0, 300)
plt.grid()
plt.legend()
plt.xlabel("Number of outlinks")
plt.title("Distribution of outlinks within the graph")
plt.show()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb = MultiLabelBinarizer()
mlb.fit(wiki_df.mid_level_categories)

In [ ]:
mlb.transform([['Culture.Architecture', 'Culture.Arts', 'Culture.Broadcasting']])

In [ ]:
mlb.classes_

In [ ]:
wiki_df.mid_level_categories[0]

In [ ]:
wiki_df.links_cleaned[:9].apply(overlap_with_titles)

In [ ]:
wiki_df.links[:5].apply(clean_links)

In [ ]:
wiki_df.links = wiki_df.links.apply(links_to_str)

In [ ]:
wiki_df